In [39]:
import requests
import json
import getpass
import pandas as pd
from pandas.io.json import json_normalize
import time

In [3]:
bearer_token = getpass.getpass()

In [8]:
headers = {
    'Authorization':'Bearer ' + bearer_token
}

In [49]:
data = dict()

In [34]:
with open('../datasets/cresci-rtbust-2019_tweets.json') as jsfile:
    cresci_rtbust_2019_data = json.load(jsfile)
cresci_rtbust_2019_data = json_normalize(cresci_rtbust_2019_data)
cresci_rtbust_2019_data.rename(columns= {'created_at': 'probe_timestamp', 'user.id': 'user-id'}, inplace=True)
cresci_rtbust_2019_data.columns = cresci_rtbust_2019_data.columns.str.replace(r'^user\.','')

In [61]:
for username in cresci_rtbust_2019_data['screen_name'][500:]:
    url = f'https://api.twitter.com/2/tweets/search/recent?query=from:{username}&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,source,text,withheld'
    response = requests.request('GET',url,headers=headers)
    tweets = response.json()
    del tweets['meta']
    data[username] = tweets

In [79]:
data2 = list()
for user in data:
    data[user]['screen_name'] = user
    if('data' in data[user]):
        data[user]['recent_tweets'] = len(data[user]['data'])
    else:
        data[user]['recent_tweets'] = 0
    data2.append(data[user])

In [80]:
with open('../tweets/cresci_rtbust_2019.json','w') as outfile:
    json.dump(data2, outfile)